In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/ECG_Heartbeat_Classification')

### 1. Import các thư viện cần thiết

Trong phần này, ta import các thư viện cơ bản để xử lý dữ liệu, xây dựng và huấn luyện mô hình học sâu (CNN) cho phân loại tín hiệu ECG.


In [ ]:
!pip install numpy==1.23.3 pandas==2.2.2 scipy==1.13.1 plotly==5.18.0 tensorflow==2.12.0

In [ ]:
!pip install scikit-learn==1.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 60.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.1.3 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.3 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.1.3 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.1.3 which is incompatible.


### 2. Tiền xử lý dữ liệu MIT-BIH (Arrhythmia Classification)

Đọc dữ liệu nhịp tim từ MIT-BIH, shuffle để tránh bias, và reshape lại đầu vào cho phù hợp với Conv1D. Nhãn là các giá trị từ 0 đến 4 tương ứng với 5 loại nhịp tim theo chuẩn AAMI EC57.


In [ ]:
import pandas as pd
import numpy as np
import random
import os
from plotly.offline import plot
import plotly.graph_objs as go
from collections import Counter

base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
image_dir = os.path.join(base_dir, "images")
os.makedirs(image_dir, exist_ok=True)

df = pd.read_csv("./input/mitbih_test.csv", header=None)
print(df.shape)
print(Counter(df[187].values))

Y = np.array(df[187].values).astype(np.int8)
X = np.array(df[list(range(187))].values)

indexes = random.sample(list(range(df.shape[0])), 10)

for i in indexes:
    data = [go.Scatter(
              x=list(range(187)),
              y=X[i, :])]

    plot({"data": data,
          "layout": {"title": "Heartbeat Class : %s " % Y[i]}}, filename=os.path.join(image_dir, f"{i}.html"))


(21892, 188)
Counter({0.0: 18118, 4.0: 1608, 2.0: 1448, 1.0: 556, 3.0: 162})


### 3. Huấn luyện mạng CNN để phân loại 5 loại nhịp tim từ tập dữ liệu MIT-BIH
Huấn luyện một mạng nơ-ron tích chập 1 chiều (1D-CNN) để phân loại các loại nhịp tim bất thường từ tập dữ liệu MIT-BIH Arrhythmia.

In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, accuracy_score


df_train = pd.read_csv("./input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("./input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 5
    #inp = Input(batch_shape=(1, 187, 1))
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()

file_path = "./models/train_mitbih_cnn.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 187, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 183, 16)           96        
                                                                 
 conv1d_1 (Conv1D)           (None, 179, 16)           1296      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 89, 16)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 89, 16)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 87, 32)            1568      
                                                             

### 4. Huấn luyện mô hình CNN để phát hiện nhồi máu cơ tim từ dữ liệu PTB-DB
Huấn luyện một mạng CNN để phân loại tín hiệu ECG bình thường và bất thường (do nhồi máu cơ tim - MI) từ tập dữ liệu PTB-DB.



In [ ]:
import pandas as pd
import numpy as np


from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

def get_model():
    nclass = 1
    #inp = Input(batch_shape=(1, 187, 1))
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "./models/train_ptbdb_cnn.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 187, 1)]          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 183, 16)           96        
                                                                 
 conv1d_9 (Conv1D)           (None, 179, 16)           1296      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 89, 16)           0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 89, 16)            0         
                                                                 
 conv1d_10 (Conv1D)          (None, 87, 32)            1568      
                                                           

### 5. Transfer Learning từ MIT-BIH sang PTB-DB: Fine-tune toàn bộ mạng để phát hiện nhồi máu cơ tim
Sử dụng biểu diễn đặc trưng đã học từ mô hình MIT-BIH và **huấn luyện lại toàn bộ mạng (fine-tuning)** với dữ liệu mới.


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    #inp = Input(batch_shape=(1, 187, 1))
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()

file_path = "./models/transfer_learning_ptbdb_full.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
model.load_weights("./models/train_mitbih_cnn.h5", by_name=True)
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 187, 1)]          0         
                                                                 
 conv1d_16 (Conv1D)          (None, 183, 16)           96        
                                                                 
 conv1d_17 (Conv1D)          (None, 179, 16)           1296      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 89, 16)           0         
 1D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 89, 16)            0         
                                                                 
 conv1d_18 (Conv1D)          (None, 87, 32)            1568      
                                                           

### 6. Transfer Learning từ MIT-BIH sang PTB-DB: Freeze phần trích đặc trưng, huấn luyện lớp phân loại
**Không huấn luyện lại (freeze) các lớp Conv1D** – tức giữ nguyên phần trích đặc trưng đã học từ bài toán phân loại nhịp tim.

**Chỉ huấn luyện lại phần đầu ra (dense layers)** để thích ứng với bài toán mới: phân loại nhịp tim bình thường (0) và nhịp tim bất thường do nhồi máu cơ tim (1).


In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", trainable=False)(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "./models/transfer_learning_ptbdb_freeze.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]
model.load_weights("./models/train_mitbih_cnn.h5", by_name=True)
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 187, 1)]          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 183, 16)           96        
                                                                 
 conv1d_9 (Conv1D)           (None, 179, 16)           1296      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 89, 16)           0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 89, 16)            0         
                                                                 
 conv1d_10 (Conv1D)          (None, 87, 32)            1568      
                                                           

### 7. Convert to file .tflite

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('./models/transfer_learning_ptbdb_full.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('./models/train_mitbih_cnn.tflite', 'wb') as f:
    f.write(tflite_model)

print("Successfully converted to .tflite")


Saved artifact at '/tmp/tmp7oh5ohi8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 187, 1), dtype=tf.float32, name='input_3')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136361725393232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361725393424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361725394768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726133840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726132304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726134608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726134032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726135376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726134800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726136144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136361726135568: TensorS

In [ ]:
import numpy as np
import tensorflow as tf

# Chuỗi dữ liệu bạn cung cấp
raw_string = """0.941 0.523 0.182 0.059 0.000 0.118 0.168 0.197 0.208 0.219 0.226 0.241 0.254 0.263 0.271 0.288 0.299 0.307 0.321 0.335 0.350 0.367 0.384 0.403 0.418 0.432 0.441 0.452 0.465 0.477 0.486 0.495 0.503 0.506 0.502 0.485 0.463 0.440 0.420 0.399 0.384 0.372 0.360 0.354 0.348 0.343 0.336 0.331 0.324 0.321 0.318 0.317 0.313 0.312 0.310 0.308 0.306 0.305 0.304 0.305 0.303 0.304 0.303 0.303 0.305 0.306 0.309 0.308 0.315 0.327 0.339 0.348 0.361 0.368 0.375 0.385 0.390 0.392 0.387 0.373 0.358 0.338 0.318 0.301 0.285 0.279 0.268 0.265 0.264 0.262 0.263 0.267 0.275 0.283 0.294 0.311 0.342 0.456 0.676 0.899 1.000 0.584 0.203 0.073 0.191 0.262 0.274 0.253 0.242 0.238 0.247 0.250 0.253 0.266 0.261 0.263 0.280 0.289 0.291 0.309 0.313 0.315 0.332 0.334 0.317 0.305 0.294 0.282 0.269 0.263 0.251 0.243 0.239 0.241 0.236 0.231 0.227 0.230 0.232 0.239 0.241 0.244 0.253 0.267 0.278 0.291 0.305 0.320 0.334 0.345 0.356 0.365 0.373 0.377 0.380 0.382 0.384 0.384 0.381 0.376 0.368 0.359 0.351 0.340 0.330 0.319 0.309 0.299 0.290 0.281 0.273 0.265 0.258 0.252 0.246 0.240 0.235 0.231 0.227 0.223 0.220 0.218 0.216 0.215 0.214 0.213 0.212"""

# Parse và reshape thành đầu vào: (1, 187, 1)
values = np.array([float(x) for x in raw_string.split()], dtype=np.float32)
input_tensor = values.reshape(1, 187, 1)


In [ ]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="train_mitbih_cnn.tflite")
interpreter.allocate_tensors()

# Lấy input/output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set input
interpreter.set_tensor(input_details[0]['index'], input_tensor)

# Run inference
interpreter.invoke()

# Lấy output
output = interpreter.get_tensor(output_details[0]['index'])
prob = float(output[0][0])
label = int(prob > 0.5)

print(f"🔍 Abnormal Probability: {prob:.4f}")
print(f"✅ Classification: {'ABNORMAL (1)' if label == 1 else 'NORMAL (0)'}")


🔍 Xác suất bất thường: 0.1094
✅ Phân loại: BÌNH THƯỜNG (0)
